In [25]:
import requests
from bs4 import BeautifulSoup
from retry import retry
import urllib
import time
import numpy as np
import pandas as pd
from datetime import date

from sqlalchemy import create_engine
from sqlalchemy import text

In [36]:
# 複数ページの情報をまとめて取得
data_samples = []

# スクレイピングするページ数
max_page = 100
# SUUMOを桜山のみ指定して検索して出力した画面のurl(ページ数フォーマットが必要)
base_url = 'https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=050&bs=040&ra=023&cb=0.0&ct=9999999&et=9999999&cn=9999999&mb=0&mt=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=&ek=323016210&rn=3230'

# リクエストがうまく行かないパターンを回避するためのやり直し
@retry(tries=3, delay=10, backoff=2)
def load_page(url):
    html = requests.get(url)
    soup = BeautifulSoup(html.content, 'html.parser')
    return soup

# 処理時間を測りたい
start = time.time()
times = []


for page in range(1, max_page + 1):
    # ページネーション用のクエリパラメータを追加
    url = f'{base_url}&page={page}'
    soup = load_page(url)
    before = time.time()
    # 物件情報リストを指定
    mother = soup.find_all(class_='cassetteitem')
        
    # 物件ごとの処理
    for child in mother:

        # 建物情報
        data_home = []
        # カテゴリ
        data_home.append(child.find(class_='ui-pct ui-pct--util1').text)
        # 建物名
        data_home.append(child.find(class_='cassetteitem_content-title').text)
        # 住所
        data_home.append(child.find(class_='cassetteitem_detail-col1').text)
        # 最寄り駅のアクセス
        children = child.find(class_='cassetteitem_detail-col2')
        for id,grandchild in enumerate(children.find_all(class_='cassetteitem_detail-text')):
            data_home.append(grandchild.text)
        # 築年数と階数
        children = child.find(class_='cassetteitem_detail-col3')
        for grandchild in children.find_all('div'):
            data_home.append(grandchild.text)

        # 部屋情報
        rooms = child.find(class_='cassetteitem_other')
        for room in rooms.find_all(class_='js-cassette_link'):
            data_room = []
            
            # 部屋情報が入っている表を探索
            for id_, grandchild in enumerate(room.find_all('td')):
                # 階
                if id_ == 2:
                    data_room.append(grandchild.text.strip())
                # 家賃と管理費
                elif id_ == 3:
                    data_room.append(grandchild.find(class_='cassetteitem_other-emphasis ui-text--bold').text)
                    data_room.append(grandchild.find(class_='cassetteitem_price cassetteitem_price--administration').text)
                # 敷金と礼金
                elif id_ == 4:
                    data_room.append(grandchild.find(class_='cassetteitem_price cassetteitem_price--deposit').text)
                    data_room.append(grandchild.find(class_='cassetteitem_price cassetteitem_price--gratuity').text)
                # 間取りと面積
                elif id_ == 5:
                    data_room.append(grandchild.find(class_='cassetteitem_madori').text)
                    data_room.append(grandchild.find(class_='cassetteitem_menseki').text)
                # url
                elif id_ == 8:
                    get_url = grandchild.find(class_='js-cassette_link_href cassetteitem_other-linktext').get('href')
                    abs_url = urllib.parse.urljoin(url,get_url)
                    data_room.append(abs_url)
            # 物件情報と部屋情報をくっつける
            data_sample = data_home + data_room
            data_samples.append(data_sample)
    
    # 1アクセスごとに1秒休む
    time.sleep(1)
    
    # 進捗確認
    # このページの作業時間を表示
    after = time.time()
    running_time = after - before
    times.append(running_time)
    print(f'{page}ページ目：{running_time}秒')
    # 取得した件数
    print(f'総取得件数：{len(data_samples)}')
    # 作業進捗
    complete_ratio = round(page/max_page*100,3)
    print(f'完了：{complete_ratio}%')
    # 作業の残り時間目安を表示
    running_mean = np.mean(times)
    running_required_time = running_mean * (max_page - page)
    hour = int(running_required_time/3600)
    minute = int((running_required_time%3600)/60)
    second = int(running_required_time%60)
    print(f'残り時間：{hour}時間{minute}分{second}秒\n')

# 処理時間を測りたい
finish = time.time()
running_all = finish - start
print('総経過時間：',running_all)

1ページ目：1.1257929801940918秒
総取得件数：68
完了：1.0%
残り時間：0時間1分51秒

2ページ目：1.15087890625秒
総取得件数：180
完了：2.0%
残り時間：0時間1分51秒

3ページ目：1.1213083267211914秒
総取得件数：250
完了：3.0%
残り時間：0時間1分49秒

4ページ目：1.108158826828003秒
総取得件数：319
完了：4.0%
残り時間：0時間1分48秒

5ページ目：1.1006529331207275秒
総取得件数：377
完了：5.0%
残り時間：0時間1分46秒

6ページ目：1.1006269454956055秒
総取得件数：439
完了：6.0%
残り時間：0時間1分45秒

7ページ目：1.1248791217803955秒
総取得件数：530
完了：7.0%
残り時間：0時間1分44秒

8ページ目：1.1032779216766357秒
総取得件数：594
完了：8.0%
残り時間：0時間1分42秒

9ページ目：1.1282460689544678秒
総取得件数：673
完了：9.0%
残り時間：0時間1分41秒

10ページ目：1.1487808227539062秒
総取得件数：785
完了：10.0%
残り時間：0時間1分40秒

11ページ目：1.098924160003662秒
総取得件数：836
完了：11.0%
残り時間：0時間1分39秒

12ページ目：1.1012341976165771秒
総取得件数：902
完了：12.0%
残り時間：0時間1分38秒

13ページ目：1.1108858585357666秒
総取得件数：970
完了：13.0%
残り時間：0時間1分37秒

14ページ目：1.099217176437378秒
総取得件数：1032
完了：14.0%
残り時間：0時間1分35秒

15ページ目：1.0875442028045654秒
総取得件数：1084
完了：15.0%
残り時間：0時間1分34秒

16ページ目：1.0989696979522705秒
総取得件数：1138
完了：16.0%
残り時間：0時間1分33秒

17ページ目：1.140890121459961秒
総取得件数：1242
完了：17.0%
残り

In [37]:
# data_samplesをDataFrameに変換
columns = ['property_type', 'building_name', 'address', 'access_1', 'access_2', 'access_3',
    'age', 'building_floors', 'room_floor', 'rent', 'management_fee',
    'deposit', 'gratuity', 'layout', 'area', 'url']
df = pd.DataFrame(data_samples, columns=columns)

# 現在の日付を掲載日として入力
current_date = date.today()
df['posting_date'] = current_date

In [38]:
df

,property_type,building_name,address,access_1,access_2,access_3,age,building_floors,room_floor,rent,management_fee,deposit,gratuity,layout,area,url,posting_date
0,賃貸アパート,地下鉄桜通線 桜山駅 2階建 築8年,愛知県名古屋市昭和区滝子通４,地下鉄桜通線/桜山駅 歩8分,地下鉄桜通線/瑞穂区役所駅 歩15分,地下鉄鶴舞線/荒畑駅 歩15分,築8年,2階建,1階,5.3万円,4000円,-,5.3万円,1SK,20.9m2,https://suumo.jp/chintai/jnc_000088214111/?bc=...,2024-02-19
1,賃貸マンション,フロイデ桜山,愛知県名古屋市昭和区菊園町４,地下鉄桜通線/桜山駅 歩11分,地下鉄鶴舞線/川名駅 歩17分,地下鉄桜通線/瑞穂区役所駅 歩19分,築6年,3階建,1階,8万円,4500円,-,8万円,1LDK,46.84m2,https://suumo.jp/chintai/jnc_000088573366/?bc=...,2024-02-19
2,賃貸マンション,アヴニール桜山,愛知県名古屋市昭和区桜山町５,地下鉄桜通線/桜山駅 歩2分,地下鉄鶴舞線/御器所駅 歩12分,地下鉄鶴舞線/荒畑駅 歩18分,築4年,8階建,3階,13.1万円,9000円,-,19.65万円,2LDK,68.67m2,https://suumo.jp/chintai/jnc_000088573368/?bc=...,2024-02-19
3,賃貸マンション,Ｅｚサイト,愛知県名古屋市昭和区大和町１,地下鉄桜通線/桜山駅 歩6分,地下鉄鶴舞線/川名駅 歩15分,地下鉄桜通線/御器所駅 歩8分,築22年,3階建,3階,17万円,-,34万円,17万円,3SLDK,97.95m2,https://suumo.jp/chintai/jnc_000076484377/?bc=...,2024-02-19
4,賃貸アパート,地下鉄桜通線 桜山駅 2階建 築8年,愛知県名古屋市瑞穂区駒場町４,地下鉄桜通線/桜山駅 歩3分,地下鉄桜通線/瑞穂区役所駅 歩11分,地下鉄鶴舞線/御器所駅 歩19分,築8年,2階建,1階,5.35万円,4000円,-,5.35万円,1K,20.13m2,https://suumo.jp/chintai/jnc_000088370773/?bc=...,2024-02-19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2081,賃貸マンション,メルヴェーユ高辻,愛知県名古屋市昭和区白金３,名鉄名古屋本線/金山駅 歩20分,地下鉄桜通線/桜山駅 歩21分,地下鉄鶴舞線/荒畑駅 歩22分,築27年,3階建,1階,4.4万円,3500円,-,-,1K,23.04m2,https://suumo.jp/chintai/jnc_000083895772/?bc=...,2024-02-19
2082,賃貸マンション,地下鉄鶴舞線 御器所駅 2階建 築51年,愛知県名古屋市昭和区長戸町１,地下鉄鶴舞線/御器所駅 歩12分,地下鉄桜通線/桜山駅 歩13分,地下鉄鶴舞線/川名駅 歩14分,築51年,2階建,2階,6.05万円,3300円,-,-,ワンルーム,39.66m2,https://suumo.jp/chintai/jnc_000088547081/?bc=...,2024-02-19
2083,賃貸一戸建て,地下鉄桜通線 瑞穂区役所駅 平屋 築85年,愛知県名古屋市瑞穂区十六町２,地下鉄桜通線/瑞穂区役所駅 歩8分,地下鉄桜通線/瑞穂運動場西駅 歩9分,地下鉄桜通線/桜山駅 歩16分,築85年,平屋,1階,18万円,-,54万円,18万円,3DK,86.49m2,https://suumo.jp/chintai/jnc_000087328342/?bc=...,2024-02-19
2084,賃貸一戸建て,長戸町4丁目貸家,愛知県名古屋市昭和区長戸町４,地下鉄鶴舞線/川名駅 歩10分,地下鉄桜通線/桜山駅 歩12分,地下鉄桜通線/御器所駅 歩14分,築87年,2階建,1-2階,12万円,-,12万円,12万円,3LDK,69.99m2,https://suumo.jp/chintai/jnc_000085151800/?bc=...,2024-02-19


In [39]:
# 重複行を削除
df_unique = df.drop_duplicates()

df_unique

,property_type,building_name,address,access_1,access_2,access_3,age,building_floors,room_floor,rent,management_fee,deposit,gratuity,layout,area,url,posting_date
0,賃貸アパート,地下鉄桜通線 桜山駅 2階建 築8年,愛知県名古屋市昭和区滝子通４,地下鉄桜通線/桜山駅 歩8分,地下鉄桜通線/瑞穂区役所駅 歩15分,地下鉄鶴舞線/荒畑駅 歩15分,築8年,2階建,1階,5.3万円,4000円,-,5.3万円,1SK,20.9m2,https://suumo.jp/chintai/jnc_000088214111/?bc=...,2024-02-19
1,賃貸マンション,フロイデ桜山,愛知県名古屋市昭和区菊園町４,地下鉄桜通線/桜山駅 歩11分,地下鉄鶴舞線/川名駅 歩17分,地下鉄桜通線/瑞穂区役所駅 歩19分,築6年,3階建,1階,8万円,4500円,-,8万円,1LDK,46.84m2,https://suumo.jp/chintai/jnc_000088573366/?bc=...,2024-02-19
2,賃貸マンション,アヴニール桜山,愛知県名古屋市昭和区桜山町５,地下鉄桜通線/桜山駅 歩2分,地下鉄鶴舞線/御器所駅 歩12分,地下鉄鶴舞線/荒畑駅 歩18分,築4年,8階建,3階,13.1万円,9000円,-,19.65万円,2LDK,68.67m2,https://suumo.jp/chintai/jnc_000088573368/?bc=...,2024-02-19
3,賃貸マンション,Ｅｚサイト,愛知県名古屋市昭和区大和町１,地下鉄桜通線/桜山駅 歩6分,地下鉄鶴舞線/川名駅 歩15分,地下鉄桜通線/御器所駅 歩8分,築22年,3階建,3階,17万円,-,34万円,17万円,3SLDK,97.95m2,https://suumo.jp/chintai/jnc_000076484377/?bc=...,2024-02-19
4,賃貸アパート,地下鉄桜通線 桜山駅 2階建 築8年,愛知県名古屋市瑞穂区駒場町４,地下鉄桜通線/桜山駅 歩3分,地下鉄桜通線/瑞穂区役所駅 歩11分,地下鉄鶴舞線/御器所駅 歩19分,築8年,2階建,1階,5.35万円,4000円,-,5.35万円,1K,20.13m2,https://suumo.jp/chintai/jnc_000088370773/?bc=...,2024-02-19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2081,賃貸マンション,メルヴェーユ高辻,愛知県名古屋市昭和区白金３,名鉄名古屋本線/金山駅 歩20分,地下鉄桜通線/桜山駅 歩21分,地下鉄鶴舞線/荒畑駅 歩22分,築27年,3階建,1階,4.4万円,3500円,-,-,1K,23.04m2,https://suumo.jp/chintai/jnc_000083895772/?bc=...,2024-02-19
2082,賃貸マンション,地下鉄鶴舞線 御器所駅 2階建 築51年,愛知県名古屋市昭和区長戸町１,地下鉄鶴舞線/御器所駅 歩12分,地下鉄桜通線/桜山駅 歩13分,地下鉄鶴舞線/川名駅 歩14分,築51年,2階建,2階,6.05万円,3300円,-,-,ワンルーム,39.66m2,https://suumo.jp/chintai/jnc_000088547081/?bc=...,2024-02-19
2083,賃貸一戸建て,地下鉄桜通線 瑞穂区役所駅 平屋 築85年,愛知県名古屋市瑞穂区十六町２,地下鉄桜通線/瑞穂区役所駅 歩8分,地下鉄桜通線/瑞穂運動場西駅 歩9分,地下鉄桜通線/桜山駅 歩16分,築85年,平屋,1階,18万円,-,54万円,18万円,3DK,86.49m2,https://suumo.jp/chintai/jnc_000087328342/?bc=...,2024-02-19
2084,賃貸一戸建て,長戸町4丁目貸家,愛知県名古屋市昭和区長戸町４,地下鉄鶴舞線/川名駅 歩10分,地下鉄桜通線/桜山駅 歩12分,地下鉄桜通線/御器所駅 歩14分,築87年,2階建,1-2階,12万円,-,12万円,12万円,3LDK,69.99m2,https://suumo.jp/chintai/jnc_000085151800/?bc=...,2024-02-19


# データの保存

In [40]:
df_unique.to_csv('scraped_data.csv')

# データベース関連の処理

In [41]:
# CSVファイルを読み込む
df = pd.read_csv('scraped_data.csv', index_col=0)

In [42]:
import configparser

config = configparser.ConfigParser()
config.read('config.ini')

db_user = config['database']['user']
db_password = config['database']['password']
db_host = config['database']['host']
db_name = config['database']['name']

In [43]:
# PostgreSQLデータベースへの接続情報
engine = create_engine(f'postgresql://{db_user}:{db_password}@{db_host}/{db_name}')

# dbを読み込み
db_df = pd.read_sql_table('rental_properties', engine)

In [44]:
# 新規レコードの挿入
## スクレイピングしたデータ（df）にあって、データベース（db_df）にないレコードを特定
new_records = df.merge(db_df, on=['url'], how='left', indicator=True).loc[lambda x : x['_merge']=='left_only']

## 不要なカラムを削除
new_records = new_records.drop(columns=['_merge'])

## 新規レコードに掲載日を設定
new_records['posting_date'] = current_date

# カラム名を修正するためのrename操作
new_records.rename(columns={
    'property_type_x': 'property_type',
    'building_name_x': 'building_name',
    'address_x': 'address',
    'access_1_x': 'access_1',
    'access_2_x': 'access_2',
    'access_3_x': 'access_3',
    'age_x': 'age',
    'building_floors_x': 'building_floors',
    'room_floor_x': 'room_floor',
    'rent_x': 'rent',
    'management_fee_x': 'management_fee',
    'deposit_x': 'deposit',
    'gratuity_x': 'gratuity',
    'layout_x': 'layout',
    'area_x': 'area',
    'url_x': 'url',
    'posting_date_x': 'posting_date'
}, inplace=True)

# 例: 不要なカラムを削除し、必要なカラムのみを選択
cleaned_df = new_records[['building_name', 'address', 'access_1', 'access_2', 'access_3', 'age', 'building_floors', 'room_floor', 'rent', 'management_fee', 'deposit', 'gratuity', 'layout', 'area', 'url', 'posting_date']].copy()

## 新規レコードをデータベースに挿入
cleaned_df.to_sql('rental_properties', engine, if_exists='append', index=False)

In [45]:
# 削除されたレコードの特定と更新
## データベースにあって、スクレイピングしたデータにないレコードを特定
removed_records = db_df.merge(df, on=['url'], how='left', indicator=True).loc[lambda x : x['_merge'] == 'left_only']

## removed_dateを更新
for index, row in removed_records.iterrows():
    update_query = text("UPDATE rental_properties SET removed_date = :current_date WHERE url = :url")
    engine.execute(update_query, current_date=current_date, url=row['url'])